# Create a Medical Assistant RAG System chat with LangChainAgent & ChromaDB


In [1]:
!pip install -q llama-cpp-python langchain langchain-community langchainhub datasets chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5

In [2]:
from datasets import load_dataset
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA , LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain import hub
from langchain_community.llms import LlamaCpp

import os

In [3]:
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader


data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')
data = data[:100]

df = pd.DataFrame(data)

df_loader = DataFrameLoader(df, page_content_column="Answer")
df_document = df_loader.load()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/233 [00:00<?, ?B/s]

medDataset_processed.csv:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

These warnings we see are because it can't perform the partition of the required size. This is because it waits for a page break to divide the text and does so when possible.

In [31]:
df[:100]

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
...,...,...,...
95,exams and tests,How to diagnose Parasites - Leishmaniasis ?,Various laboratory methods can be used to diag...
96,treatment,What are the treatments for Parasites - Leishm...,"Before considering treatment, the first step i..."
97,prevention,How to prevent Parasites - Leishmaniasis ?,No vaccines or drugs to prevent infection are ...
98,information,What is (are) Parasites - Toxoplasmosis (Toxop...,A single-celled parasite called Toxoplasma gon...


In [4]:
# split into chunks
text_splitter = CharacterTextSplitter(chunk_size=1250, separator="\n", chunk_overlap=100)
texts = text_splitter.split_documents(df_document)

In [5]:
# create embeddings and vector store in chroma
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
chroma_db = Chroma.from_documents(texts, embedding_model, persist_directory="./chromadb")


<ipython-input-5-9df216fdfb5f>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
!pip install -q huggingface_hub

from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    filename="mistral-7b-instruct-v0.1.Q4_K_M.gguf"
)


mistral-7b-instruct-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

In [7]:
# load mistral with llama-cpp
llm = LlamaCpp(
    model_path=model_path,
    temperature=0.0,
    max_tokens=1024,
    top_p=1,
    n_ctx=4096,
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.1-GGUF/snapshots/731a9fc8f06f5f5e2db8a0cf9d256197eb6e05d1/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv

In [8]:
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType

# Agent Use cases:

In general, using LangChain agents is useful when there is a need to manage and coordinate multiple tools and processing steps. In summary, the situations where LangChain agents are used include:

- Multi-step responses: When questions or tasks involve multiple steps to solve (e.g., combining medical Q&A with summarizing content).
  
- Utilizing diverse tools: When you want to seamlessly integrate multiple tools like RAG, summarization, or database searches.
  
- Managing dynamic interactions: When answers need to be adjusted based on conversation history and changing conditions.
  
- Coordinating complex processes: In projects that require the agent to autonomously decide which tools to use in order to quickly arrive at comprehensive results.

In [32]:
# tool1 = RAG
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=chroma_db.as_retriever())

# tool2 = summarizer
summary_prompt = PromptTemplate.from_template("Summarize this text:\n\n{text}")
summary_chain = LLMChain(llm=llm, prompt=summary_prompt)

In [40]:
# create memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4,
    return_messages=True)

tools = [
    Tool(
        name="Medical QA",
        func=qa.run,
        description="Answering medical questions. Example: What are the symptoms of high blood pressure?"
    ),
    Tool(
        name="Summarizer",
        func=summary_chain.run,
        description="Summarizing a text. Example: Summarize this text..."
    )
]

In [41]:
# Create agent using `initialize_agent` which handles prompt construction and memory
agent_executor = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,  # Using a conversational agent for context management
    memory=conversational_memory,
    verbose=True
)

<ipython-input-41-67d329a90f6a>:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [42]:
# Use agent to handle user input and generate response
response = agent_executor.invoke({
    "input": "What are the symptoms of Lymphocytic Choriomeningitis?"
})

print(response)



> Entering new AgentExecutor chain...


Llama.generate: 1 prefix-match hit, remaining 460 prompt tokens to eval
llama_perf_context_print:        load time =  104922.96 ms
llama_perf_context_print: prompt eval time =  243645.17 ms /   868 tokens (  280.70 ms per token,     3.56 tokens per second)
llama_perf_context_print:        eval time =   22370.89 ms /    35 runs   (  639.17 ms per token,     1.56 tokens per second)
llama_perf_context_print:       total time =  151935.14 ms /   903 tokens
Llama.generate: 1 prefix-match hit, remaining 565 prompt tokens to eval



Thought: Do I need to use a tool? Yes
Action: Medical QA
Action Input: What are the symptoms of asthma?

llama_perf_context_print:        load time =  104922.96 ms
llama_perf_context_print: prompt eval time =  148514.68 ms /   565 tokens (  262.86 ms per token,     3.80 tokens per second)
llama_perf_context_print:        eval time =   50933.90 ms /    80 runs   (  636.67 ms per token,     1.57 tokens per second)
llama_perf_context_print:       total time =  199535.65 ms /   645 tokens
Llama.generate: 1 prefix-match hit, remaining 581 prompt tokens to eval



Observation:  Asthma is a chronic respiratory condition that causes inflammation and narrowing of the airways. The symptoms of asthma include wheezing, shortness of breath, coughing, especially at night or early in the morning, and difficulty breathing. These symptoms can vary from person to person and may be worse during certain times, such as exercise or exposure to allergens.
Thought:

llama_perf_context_print:        load time =  104922.96 ms
llama_perf_context_print: prompt eval time =  148304.17 ms /   581 tokens (  255.26 ms per token,     3.92 tokens per second)
llama_perf_context_print:        eval time =   58781.38 ms /    92 runs   (  638.93 ms per token,     1.57 tokens per second)
llama_perf_context_print:       total time =  207187.94 ms /   673 tokens


 Do I need to use a tool? No
AI: Asthma is a chronic respiratory condition that causes inflammation and narrowing of the airways. The symptoms of asthma include wheezing, shortness of breath, coughing, especially at night or early in the morning, and difficulty breathing. These symptoms can vary from person to person and may be worse during certain times, such as exercise or exposure to allergens.

> Finished chain.
{'input': 'What are the symptoms of asthma?', 'chat_history': [], 'output': 'Asthma is a chronic respiratory condition that causes inflammation and narrowing of the airways. The symptoms of asthma include wheezing, shortness of breath, coughing, especially at night or early in the morning, and difficulty breathing. These symptoms can vary from person to person and may be worse during certain times, such as exercise or exposure to allergens.'}


To make queries we simply call the `agent` directly.

First i will try a order not related to the Medical field.

In [43]:
agent_executor.invoke({"input": "Give me the area of square of 2x2"})



> Entering new AgentExecutor chain...


Llama.generate: 445 prefix-match hit, remaining 141 prompt tokens to eval
llama_perf_context_print:        load time =  104922.96 ms
llama_perf_context_print: prompt eval time =   38052.32 ms /   141 tokens (  269.87 ms per token,     3.71 tokens per second)
llama_perf_context_print:        eval time =   19079.62 ms /    30 runs   (  635.99 ms per token,     1.57 tokens per second)
llama_perf_context_print:       total time =   57163.01 ms /   171 tokens



Thought: Do I need to use a tool? No
AI: The area of a square with side length 2 is 4.

> Finished chain.


{'input': 'Give me the area of square of 2x2',
 'chat_history': [HumanMessage(content='What are the symptoms of asthma?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Asthma is a chronic respiratory condition that causes inflammation and narrowing of the airways. The symptoms of asthma include wheezing, shortness of breath, coughing, especially at night or early in the morning, and difficulty breathing. These symptoms can vary from person to person and may be worse during certain times, such as exercise or exposure to allergens.', additional_kwargs={}, response_metadata={})],
 'output': 'The area of a square with side length 2 is 4.'}

Perfect, the model has responded without accessing the configured knowledge database.

Now I will try with a question that is also not related to health.

In [44]:
agent_executor.invoke({"input": "Do you know who is Clark Kent?"})



> Entering new AgentExecutor chain...


Llama.generate: 567 prefix-match hit, remaining 73 prompt tokens to eval
llama_perf_context_print:        load time =  104922.96 ms
llama_perf_context_print: prompt eval time =   19660.66 ms /    73 tokens (  269.32 ms per token,     3.71 tokens per second)
llama_perf_context_print:        eval time =   29083.56 ms /    45 runs   (  646.30 ms per token,     1.55 tokens per second)
llama_perf_context_print:       total time =   48792.93 ms /   118 tokens



Thought: Do I need to use a tool? No
AI: Yes, Clark Kent is the alter ego of Superman. He is a journalist and reporter for the Daily Planet newspaper in Metropolis.

> Finished chain.


{'input': 'Do you know who is Clark Kent?',
 'chat_history': [HumanMessage(content='What are the symptoms of asthma?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Asthma is a chronic respiratory condition that causes inflammation and narrowing of the airways. The symptoms of asthma include wheezing, shortness of breath, coughing, especially at night or early in the morning, and difficulty breathing. These symptoms can vary from person to person and may be worse during certain times, such as exercise or exposure to allergens.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Give me the area of square of 2x2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The area of a square with side length 2 is 4.', additional_kwargs={}, response_metadata={})],
 'output': 'Yes, Clark Kent is the alter ego of Superman. He is a journalist and reporter for the Daily Planet newspaper in Metropolis.'}

It has not accessed either, as the model has been able to identify that it is not a question related to the database that LangChain provides.

Now it's time to try with a question related to Medicine. Let's see if the model can understand that it should first look for information in the vector database at its disposal.

In [45]:
 agent_executor.memory.clear()

In [46]:
agent_executor.invoke({"input": """I have a patient that can have Botulism,
how can I confirm the diagnosis?"""})



> Entering new AgentExecutor chain...


Llama.generate: 445 prefix-match hit, remaining 26 prompt tokens to eval
llama_perf_context_print:        load time =  104922.96 ms
llama_perf_context_print: prompt eval time =    8161.84 ms /    26 tokens (  313.92 ms per token,     3.19 tokens per second)
llama_perf_context_print:        eval time =   18871.60 ms /    30 runs   (  629.05 ms per token,     1.59 tokens per second)
llama_perf_context_print:       total time =   27064.40 ms /    56 tokens
Llama.generate: 1 prefix-match hit, remaining 746 prompt tokens to eval



Thought: Do I need to use a tool? Yes
Action: Medical QA
Action Input: Botulism diagnosis

llama_perf_context_print:        load time =  104922.96 ms
llama_perf_context_print: prompt eval time =  196772.86 ms /   746 tokens (  263.77 ms per token,     3.79 tokens per second)
llama_perf_context_print:        eval time =  214668.45 ms /   332 runs   (  646.59 ms per token,     1.55 tokens per second)
llama_perf_context_print:       total time =  411892.02 ms /  1078 tokens
Llama.generate: 1 prefix-match hit, remaining 838 prompt tokens to eval



Observation:  Botulism diagnosis is a complex process that requires a combination of clinical symptoms, laboratory tests, and sometimes imaging studies. Physicians may consider the diagnosis if the patient's history and physical examination suggest botulism, but these clues are usually not enough to allow a definitive diagnosis. Other diseases such as Guillain-Barré syndrome, stroke, and myasthenia gravis can appear similar to botulism, and special tests may be needed to exclude these other conditions. These tests may include a brain scan, spinal fluid examination, nerve conduction test (electromyography, or EMG), and a tensilon test for myasthenia gravis. Tests for botulinum toxin and for bacteria that cause botulism can be performed at some state health department laboratories and at CDC. The classic symptoms of botulism include double vision, blurred vision, drooping eyelids, slurred speech, difficulty swallowing, dry mouth, and muscle weakness. Infants with botulism appear letharg

llama_perf_context_print:        load time =  104922.96 ms
llama_perf_context_print: prompt eval time =  214622.65 ms /   838 tokens (  256.11 ms per token,     3.90 tokens per second)
llama_perf_context_print:        eval time =  223781.22 ms /   347 runs   (  644.90 ms per token,     1.55 tokens per second)
llama_perf_context_print:       total time =  438882.54 ms /  1185 tokens


 Do I need to use a tool? No
AI: Botulism diagnosis is a complex process that requires a combination of clinical symptoms, laboratory tests, and sometimes imaging studies. Physicians may consider the diagnosis if the patient's history and physical examination suggest botulism, but these clues are usually not enough to allow a definitive diagnosis. Other diseases such as Guillain-Barré syndrome, stroke, and myasthenia gravis can appear similar to botulism, and special tests may be needed to exclude these other conditions. These tests may include a brain scan, spinal fluid examination, nerve conduction test (electromyography, or EMG), and a tensilon test for myasthenia gravis. Tests for botulinum toxin and for bacteria that cause botulism can be performed at some state health department laboratories and at CDC. The classic symptoms of botulism include double vision, blurred vision, drooping eyelids, slurred speech, difficulty swallowing, dry mouth, and muscle weakness. Infants with botul

{'input': 'I have a patient that can have Botulism,\nhow can I confirm the diagnosis?',
 'chat_history': [],
 'output': "Botulism diagnosis is a complex process that requires a combination of clinical symptoms, laboratory tests, and sometimes imaging studies. Physicians may consider the diagnosis if the patient's history and physical examination suggest botulism, but these clues are usually not enough to allow a definitive diagnosis. Other diseases such as Guillain-Barré syndrome, stroke, and myasthenia gravis can appear similar to botulism, and special tests may be needed to exclude these other conditions. These tests may include a brain scan, spinal fluid examination, nerve conduction test (electromyography, or EMG), and a tensilon test for myasthenia gravis. Tests for botulinum toxin and for bacteria that cause botulism can be performed at some state health department laboratories and at CDC. The classic symptoms of botulism include double vision, blurred vision, drooping eyelids, s

Perfect, the most important thing for us is that it has been able to identify that it should go to the medical database to search for information about the symptoms.

In [47]:
agent_executor.invoke({"input": "Is this an important illness?"})



> Entering new AgentExecutor chain...


Llama.generate: 445 prefix-match hit, remaining 401 prompt tokens to eval
llama_perf_context_print:        load time =  104922.96 ms
llama_perf_context_print: prompt eval time =  105356.57 ms /   401 tokens (  262.73 ms per token,     3.81 tokens per second)
llama_perf_context_print:        eval time =  129016.39 ms /   200 runs   (  645.08 ms per token,     1.55 tokens per second)
llama_perf_context_print:       total time =  234620.54 ms /   601 tokens



Thought: Do I need to use a tool? No
AI: Yes, it is an important illness. Botulism is a rare but serious illness that can be life-threatening if left untreated. It is caused by the bacterial toxin produced by Clostridium botulinum and can occur in both food and nonfood sources. Symptoms typically begin with double vision, blurred vision, drooping eyelids, slurred speech, difficulty swallowing, dry mouth, and muscle weakness. Infants with botulism appear lethargic, feed poorly, are constipated, and have a weak cry and poor muscle tone. These symptoms can progress to cause paralysis of the respiratory muscles, arms, legs, and trunk. If left untreated, botulism can be fatal. It is important for healthcare providers to diagnose and treat botulism promptly to prevent serious complications and death.

> Finished chain.


{'input': 'Is this an important illness?',
 'chat_history': [HumanMessage(content='I have a patient that can have Botulism,\nhow can I confirm the diagnosis?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Botulism diagnosis is a complex process that requires a combination of clinical symptoms, laboratory tests, and sometimes imaging studies. Physicians may consider the diagnosis if the patient's history and physical examination suggest botulism, but these clues are usually not enough to allow a definitive diagnosis. Other diseases such as Guillain-Barré syndrome, stroke, and myasthenia gravis can appear similar to botulism, and special tests may be needed to exclude these other conditions. These tests may include a brain scan, spinal fluid examination, nerve conduction test (electromyography, or EMG), and a tensilon test for myasthenia gravis. Tests for botulinum toxin and for bacteria that cause botulism can be performed at some state health department laboratorie